## Scraping reviews using Selenium

Here is another example of how Selenium can be used to interact with websites making use of Ajax (Asynchronous JavaScript):

### Selenium is a chrome automation framework

It will enable us to tell chrome:
* go to page bbc.co.uk/weather
* "click the work 'next'"
* scroll down

Selenium will basically open a simplified version of Chrome, for a few seconds, use it and close it afterwards. You might even see it flash on your screen quickly. Then we will use beautiful soup to understand the code.

### BeautifulSoup is an HTML parsing framework

It will enable us to:
* copy the html of the tags eg. div, table
* extract text from these tags

## Getting selenium (don't skip this!)-- You need to download the chromedrive by yourself.

1. find out which version of chrome you have, in chrome open page: chrome://settings/help
2. Go to the list of selenium versions and find folder with yoru version (eg. 87.0.4280.88) https://chromedriver.storage.googleapis.com/index.html
3. Go into the folder for your version and download the zip file with the version for your operating system (most likely `chromedriver_mac64.zip` or `chromedriver_win32.zip` ).
4. unzip that file on yoru machine and put it in the folder where this notebook is. unzipped file will be called `chromedriver` or `chromedriver.exe`.

In [1]:
!pip install selenium

In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [3]:
# define method that will create a browser, suitable to your operating system
import sys
def get_a_browser():
    if sys.platform.startswith('win32') or sys.platform.startswith('cygwin'):
        return webdriver.Chrome() # windows
    else:
        return webdriver.Chrome('chromedriver') # mac

**Important Note**: allowing your system to run `chromedriver`. This needs to be done just once.

If you are on a mac, you will need to allow your system to use chromium. Run below cell, and you will likely see a warning the first time, click 'cancel' (don't click 'Delete').

After you see the warning, go into `Settings > Security&Privacy > General` and `"Allow Anyway"`.

On a pc the process will be simpler. When asked you'll need to allow computer to use the `chromedriver.exe` file.

## Task: let's try to scrape an interactive website

What will be the weather in Edinburgh in 2 days?

You need a web browser, pen and paper!

In this task you will be asked to do something by yourself (using your web browser, mouse and keyboard), and then you will see how you cen program `Selenium` to do it for you.

**Use www.bbc.co.uk/weather to find out what time will be the sunrise in EDINBURGH next Sunday.**

Do it at least 3 times and observe all the steps you are taking. Make a very detailed list of all the steps, as if you had to describe them to someone over the phone without seeing their screen. See example below.

it will look a bit like this:
* ok, go to www.bbc.co.uk/weather and wait for it to load
* scroll down, do you see a link with words 'Edinburgh' on it? Click it.
* Wait a minute for it to load.
* ok, now scroll down and ...

When you are done with this exercise, we will try to instruct Selenium (Chrome automation tool) to do it for us. Do you think you can try to use Chrome Dev tools to make yoru steps more specific? eg. Instead of saying "copy text in that bold link next to the word Sunrise" try to say "copy text from the html span item with a class `wr-c-astro-data__time`".

**SERIOUSLY: Take a few minutes to do this. It will make you learn more from the below code!**

Ok. And now let's get the python to do it for us.

In [6]:
browser = get_a_browser()

# the url we want to open
url = u'https://www.bbc.co.uk/weather'

# the browser will start and load the webpage
browser.get(url)

# we wait 1 second to let the page load everything
time.sleep(1)

# we search for an element that is called 'customer reviews', which is a button
# the button can be clicked with the .click() function
browser.find_element(By.LINK_TEXT,"Edinburgh").click();

# sleep again, let everything load
time.sleep(1)

# we load the HTML body (the main page content without headers, footers, etc.)
body = browser.find_element(By.TAG_NAME,'body')

# we use seleniums' send_keys() function to physically scroll down where we want to click
body.send_keys(Keys.PAGE_DOWN)

# search for the next button to access the next reviews
try:
    # link will look like "Sun 12Dec" so we use find_element_by_partial_link_text()
    next_button = browser.find_element(By.PARTIAL_LINK_TEXT,'Sun ') 
    next_button.click()
except NoSuchElementException:  #if such element does not exist, just stop looping
    print("something went wrong. There was no Sunday link.")
    
# load current view of the page into a soup
soup = BeautifulSoup(browser.page_source, 'html.parser')

"""
1. Find all the elements of class pros and print them 
2. These values include today's sunrise and sunset time, and the following 13 days.
3. `browser.page_source` always get the whole page, so we can only find all
4. A not smart, but workable solution is to count how many days between today and next sunday 
   and then choose the right element of all sunrise_tag list.
"""
# The whole list
sunrise_tag = soup.find_all("span", {"class" : 'wr-c-astro-data__time'})
# How many days between today and the next sunday
diff = int(next_button.get_attribute('id')[-1])

print("Sunrise next Sunday: ", sunrise_tag[2*diff].text)

Sunrise next Sunday:  08:01


In [13]:
for i in range(8):
    print(i, sunrise_tag[i].text)

0 08:32
1 16:15
2 08:31
3 16:17
4 08:29
5 16:18
6 08:28
7 16:20


## Using API to access Twitter

Tweepy is a library that interfaces with the Twitter API:

In [15]:
!pip install tweepy
import tweepy
# weeeply is a python library for accessing twitter data via twitter API. 
# # below I am sharing my demo credenmtials, they will work for testing it,
# but for your project you'll need to create  your own credentials.
# - create a twitter app with your twitter avound (one per group will do) https://developer.twitter.com/en/apps
# - follow the tutorial on tweepy to set it up https://tweepy.readthedocs.io/

     |████████████████████████████████| 101 kB 2.6 MB/s ta 0:00:01
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
     |████████████████████████████████| 151 kB 2.7 MB/s eta 0:00:01


In [16]:
Bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMi%2BYAEAAAAA%2F2LLeju%2BgWlNK34g6PMT14scXzQ%3DHa0gE8PJoBnMVlnyoC3648USErcR6E86QadKgbKlBMIrKVNiYz'  # please generate it from twitter developer by yourself and put it here
client = tweepy.Client(Bearer_token)

In [17]:
for tweet in tweepy.Paginator(client.search_recent_tweets, "University of Edinburgh",
                              max_results=100).flatten(limit=10):
    print(tweet.text)

Looking for Chinese New Year events? Look no further than our Eventbrite: https://t.co/dvChrFbzW3

#ChineseNewYear2023 #YearOfTheRabbit #China #culture #museum #EVENT #ceilidh #dance #family #children #crafting #scavengerhunt #museum #activity
Master’s Degree Scholarships by the MasterCard Foundation at the University of Edinburgh. 

Eligible for both online and on campus study. 

Follow the link for more details:  https://t.co/kLkFEXqxNb https://t.co/inzs7MxHVG
RT @RoyalCentral: The Princess Royal, as Chancellor of the University of Edinburgh, will visit the FastBlade Structural Testing facility in…
RT @AHFdoco: @KayBurley @elliebgomersall Our Film Adult Human Female was cancelled by student trans rights activists at Edinburgh Universit…
RT @RoyalCentral: The Princess Royal, as Chancellor of the University of Edinburgh, will visit the FastBlade Structural Testing facility in…
We had the pleasure of hosting @geraldchirinda and @munangenda from the @FutureAfriForum at the University of 

**More details, please refer to https://developer.twitter.com/en/docs/twitter-api/tweets/search/api-reference/get-tweets-search-recent**

## Scraping tweets with Selenium

In this exercise we will use selenium to copy-paste some tweets straight from the twitter website.

Be aware that there are terms and conditions about how you can use these coppied data. If you abuse or overuse scraping, twitter might block or throttle (slow down) your access to their site. (like, don't scrpate 1000s of tweets in 100 parrallel selenium windows).

This time, we import selenium first:

In [19]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [20]:
# define method that will create a browser, suitable to your operating system
import sys
def get_a_browser():
    if sys.platform.startswith('win32') or sys.platform.startswith('cygwin'):
        return webdriver.Chrome() # windows
    else:
        return webdriver.Chrome('chromedriver') # mac

The webdriver object can launch Internet Explorer, Firefox, and Chrome. Despite your preference, the ChromeDriver (which is a light version of Chrome) is the most widely used and complete one. You can use it to start a twitter page:

In [21]:
# launch the browser
browser = get_a_browser()

# launch the Twitter search page
twitter_url = u'https://twitter.com/search?q='

# Add the search term
query = u'%40edinburgh'
# note: %40 is a code for @ symbol, so we're asking for the tweets with @edinburgh

# Create the url
url = twitter_url+query

# Get the page
browser.get(url)

Let's do this again and unleash the power of Selenium by using keyboard controls to manipulate a page:

In [22]:
browser = get_a_browser()
browser.get(url)

# Let the Tweets load
time.sleep(1)

# Find the body of the HTML page
body = browser.find_element(By.TAG_NAME,'body')

# Keep scrolling down using a simulation of the PAGE_DOWN button
for _ in range(5):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    
# Get the tweets scores by their class (similar to Beautifulsoup's find())
retweets = browser.find_elements(By.XPATH,"//div[@data-testid='retweet']");

print("number of tweets scraped: ", len(retweets))

# Print Tweets
for retweet in retweets:
    print("\n--NEXT TWEET---\n", retweet.text, "\n-----\n")

number of tweets scraped:  4

--NEXT TWEET---
 151 
-----


--NEXT TWEET---
 102 
-----


--NEXT TWEET---
 16 
-----


--NEXT TWEET---
 38 
-----

